# PART 2. 데이터 입출력
## 1. 외부 파일 읽어오기
### 1-1. CSV 파일 : read_csv()


In [4]:
# -*- coding: utf-8 -*-

# 라이브러리 불러오기
import pandas as pd

# 파일경로를 찾고, 변수 file_path에 저장
file_path = './counterparty_dict.csv'

# read_csv() 함수로 데이터프레임 변환. 변수 df1에 저장
df1 = pd.read_csv(file_path)
#print(df1)
#print('\n')

# read_csv() 함수로 데이터프레임 변환. 변수 df2에 저장. header=None 옵션
df2 = pd.read_csv(file_path, header=None)
#print("df2",df2)
#print('\n')

# read_csv() 함수로 데이터프레임 변환. 변수 df3에 저장. index_col=None 옵션
df3 = pd.read_csv(file_path, index_col=None)
print("df3",df3)
print('\n')

# read_csv() 함수로 데이터프레임 변환. 변수 df4에 저장. index_col='c0' 옵션
df4 = pd.read_csv(file_path, index_col='LEI')
print("df4",df4)

df3    counterparty_name kr_counterparty_name  \
0             NH농협은행            농협은행 주식회사   
1            KDB산업은행               한국산업은행   
2          메리츠화재해상보험         메리츠화재해상보험(주)   
3               신한은행                 신한은행   
4            IBK연금보험         아이비케이연금보험(주)   
5            한국수출입은행              한국수출입은행   
6               우리은행             (주) 우리은행   
7             KB국민은행                 국민은행   
8               하나은행            주식회사 하나은행   
9              현대캐피탈            현대캐피탈주식회사   
10            한화생명보험           한화생명보험 (주)   
11            KB손해보험         주식회사 케이비손해보험   
12             현대차증권             현대차증권(주)   
13            NH투자증권          엔에이치투자증권(주)   
14          현대해상화재보험         현대해상화재보험 (주)   
15            한국투자증권          한국투자증권 주식회사   
16           코리안리재보험         코리안리재보험 주식회사   
17            동양생명보험            동양생명보험(주)   
18          미래에셋생명보험         미래에셋생명보험주식회사   
19            한국자금중개           한국자금중개주식회사   
20           KDB생명보험          케이디비생명보험(주)   
21    

csv 파일에 따라서는 쉼표 대신 탭이나 공백으로 텍스트를 구분하기도 한다. 이때는 구분자 옵션에 알맞게 입력해야 한다.
### read_csv() 함수의 옵션
* path: 파일의 경로

* sep(혹은 delimeter): 텍스트 데이터를 필드별로 구분하는 문자

* header: 열 이름으로 사용할 행의 번호(기본값은 0). 헤더가 없고 첫 행부터 데이터가 있는 경우 None으로 설정 가능

* index_col: 행 인덱스로 사용할 열의 번호 혹은 이름

* names: 열 이름으로 사용할 문자열의 리스트

* skiprows: 처음 몇 줄을 skip할 것인지 설정(숫자 입력), 리스트로 설정 가능

* parse_dates: 날짜 텍스트를 datetime64로 변환할 것인지 설정(기본값 False)

* skip_footer: 마지막 몇 줄을 스킵할 것인지 설정(숫자 입력)

* encoding: 텍스트 인코딩 종류를 지정

### 1-2. Excel 파일 : read_excel()
### 1-3. Json 파일 : read_json()

## 2. 웹에서 가져오기
### 2-1. HTML 웹 페이지에서 표 속성 가져오기 : read_html()
### 2-2. 웹 스크래핑


In [9]:
#주민번호의 맨 뒷자리를 모두 *표 처리 하고 싶을 때? 정규표현식을 사용하면 식이 짧아진다.
import re 

data = """
park 800905-1049118
kim  700905-1059119
"""

pat = re.compile("(\d{6})[-]\d{7}")
print(pat.sub("\g<1>-*******", data))


park 800905-*******
kim  700905-*******



### 메타 문자 . ^ $ * + ? { } [ ] \ | ( )

정규 표현식을 위한 메타문자

* ^ :  문자열의 맨 처음과 일치함을 의미

* $ :  문자열의 끝과 매치함을 의미
* \b : 단어 구분자(Word boundary)

* () : 그루핑

기타 : https://wikidocs.net/4309 참고

In [8]:
from bs4 import BeautifulSoup
import requests
import re # 정규 표현식을 쓸 때 사용하는 파이썬 내장 모듈: match(), fullmatch(), findall(), search() 등.
import pandas as pd

# 위키피디아 미국 ETF 웹 페이지에서 필요한 정보를 스크래핑하여 딕셔너리 형태로 변수 etfs에 저장
url = "https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'lxml')   
rows = soup.select('div > ul > li')
    
etfs = {}
for row in rows:
    
    try:
        etf_name = re.findall('^(.*) \(NYSE', row.text) # findall() : 정규식과 매치되는 모든 문자열(substring)을 리스트로 돌려준다.
        etf_market = re.findall('\((.*)\|', row.text)
        etf_ticker = re.findall('NYSE Arca\|(.*)\)', row.text)
        
        if (len(etf_ticker) > 0) & (len(etf_market) > 0) & (len(etf_name) > 0):
            etfs[etf_ticker[0]] = [etf_market[0], etf_name[0]]

    except AttributeError as err:
        pass    

# etfs 딕셔너리 출력
#print(etfs)
print('\n')

# etfs 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame(etfs)
print(df)



                                  ITOT                         IWV  \
0                            NYSE Arca                   NYSE Arca   
1  iShares Core S&P Total US Stock Mkt  iShares Russell 3000 Index   

                         SCHB                                            FNDB  \
0                   NYSE Arca                                       NYSE Arca   
1  Schwab US Broad Market ETF  Schwab Fundamental U.S. Broad Market Index ETF   

                           VT                          VTI  \
0                   NYSE Arca                    NYSE Arca   
1  Vanguard Total World Stock  Vanguard Total Stock Market   

                                 VXUS                   VTHR  \
0                           NYSE Arca              NYSE Arca   
1  Vanguard Total International Stock  Vanguard Russell 3000   

                        DIA                              RSP  ...  \
0                 NYSE Arca                        NYSE Arca  ...   
1  DIAMONDS Trust, Series

## 3. API를 활용하여 데이터 수집하기
## 4. 데이터 저장하기
### 4-1. CSV 파일로 저장 : to_csv()
### 4-2. JSON 파일로 저장 : to_json()
### 4-3. Excel 파일로 저장 : to_excel() 
to_excel() 함수 내 sheet_name="" 으로 시트 이름 설정 가능
### 4-4. 여러 개의 데이터프레임을 하나의 excel 파일로 저장 : ExcelWriter()
to_excel() 함수 내 sheet_name="" 으로 시트 이름 설정 가능
